In [1]:
from configparser import ConfigParser
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np
from numpy import matrix
import pickle
from sklearn.metrics import mean_squared_error
from get_timbre import get_timbre
from evaluation import evaluate_data
import tensorflow as tf

parser = ConfigParser()
parser.read('./spotify_credentials.cfg')

SPOTIPY_CLIENT_ID = parser.get('spotify', 'SPOTIPY_CLIENT_ID')
SPOTIPY_CLIENT_SECRET = parser.get('spotify', 'SPOTIPY_CLIENT_SECRET')

sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET))

user_id = 'czdoifmfngjhvoetavlok9dg5'

In [2]:
playlist_ids = ['5odVaQ10ISGhvuakjVGmxp', '0mCV9EHiVXEsbw307lBF25', '4VGvI1dODxGyBop5jFhgfN',
                '5pVRL48wvxNRL8lWkxdVCX', '5Kq61WYIbYGLB6dvuTKzZV', '3MO4sQXwvB13TAdiSoG9AR',
                '64iSau8h74DkOUG8JVY3W2', '5ZWw4qko5ccxnEIA8blVSq', '35iVxmwsHyJVSdvRasuK8V',
                '06LB9aYKvibLEXb4Qf3gHM', '2dfxSJXVFYpZi2YOjgbwRX', '6FwDZbpBnGXBOJ1UwW9lKb',
                '44vN1eprBxUdirtb9pCOge', '64mSbvFQYIW547z4zWrEQK', '3VBWC197hTEmayqbDTmvp4',
                '2Am8cD0icf62d4dL7NrDTx', '0fjjY976NVJ03tPhxjamcX', '0vzdOGXJBf6SBFeYbwfpnL',
                '5JufAIzUR25lgE8zrums4k', '2ppGD3RwccxuPSJZiyZrCm', '4LBHOUvGlk4RtaMbfB5huK',
                '1VJZKBsulYCry6QpSoW5mG', '0dnsYgOOtfsOg78qLrZEAv', '5KtPLkVfM18fVQ4puapK98',
                '51eXnQHgNcFw7xGllU5kCf', '23x3CI4Cncf6BpSnIeZlF7', '1rDhjtHJkAIlUWgJlxQSRF',
                '5cS1vW9gzr2lLFZ0XpkmfD', '2xxXRaeMZSoAiUslTqbB7S', '0Gx1JfJVTkWSfCcTkMgJiA',
                ]

# playlist_ids = ['5odVaQ10ISGhvuakjVGmxp']

In [3]:
# Get all track data (has been run, no longer needed)

# x = 1
# for playlist_id in playlist_ids:
#     print('Running playlist ' + playlist_id)

#     # Get track data
    
#     results = sp.playlist_tracks(playlist_id)
#     tracks = results['items']
#     track_ids = []
#     for track in tracks:
#         track_ids.append(track['track']['id'])

#     columns_to_remove = ['analysis_url', 'type', 'uri', 'track_href']
#     headings = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
#                 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
#                 'id', 'duration_ms', 'time_signature', 'track_title', 'album_title',
#                 'album_artist', 'track_number', 'total_tracks']

#     playlist_data = pd.DataFrame(columns=headings)
#     playlist_length = len(track_ids)

#     i = 0
#     for track in track_ids:
#         results = sp.audio_features(track)
#         if results[0]: 
#             features = results[0]
        
#         features_matrix = pd.DataFrame.from_records(features, index=[0])
        
#         # Remove unneeded columns
#         features_matrix.drop(columns = columns_to_remove, axis = 1, inplace = True)

#         # Add track info
#         features_matrix['track_title'] = tracks[i]['track']['name']
#         features_matrix['album_title'] = tracks[i]['track']['album']['name']
#         features_matrix['album_artist'] = tracks[i]['track']['album']['artists'][0]['name']

#         # Add track number and total tracks
#         features_matrix['track_number'] = tracks[i]['track']['track_number']
#         features_matrix['total_tracks'] = playlist_length

#         playlist_data = pd.concat([playlist_data, features_matrix])
#         i += 1

#     playlist_data = playlist_data.reset_index(drop=True)
#     playlist_data_full = pd.DataFrame(playlist_data)

#     # Get timbre data

#     timbre = pd.DataFrame(columns = ['song_timbre', 'song_timbre_start', 'song_timbre_end', 'loudness_start', 'loudness_end'])

#     for track in track_ids:
#         print('Getting timbre for: '+ track)
#         track_timbre = get_timbre(track, playlist_data_full)
#         timbre = pd.concat([timbre, track_timbre], axis=0)

#     timbre = timbre.reset_index(drop=True)
#     playlist_data = pd.concat([playlist_data, timbre], axis=1)

#     playlist_data.to_csv('./data/test_set/playlist_data_' + str(x) + '.csv')
#     playlist_data_full.to_csv('./data/test_set/playlist_data_full_' + str(x) + '.csv')
#     x += 1

In [ ]:
# Evalute all tracks

x = 1
sorted_evals = []
original_evals = []
mean_squared_errors = []
eval_changes = []

for playlist_id in playlist_ids:
    
    print('Running playlist ' + playlist_id)

    # Read and set up data
    playlist_data = pd.read_csv('./data/test_set/playlist_data_' + str(x) + '.csv')
    playlist_data.drop(columns = ['Unnamed: 0'], axis = 1, inplace = True)
    playlist_data_full = playlist_data.copy(deep=True)
    x += 1

    # Remove unnecessary testing columns
    columns_to_remove = ['id', 'track_title', 'album_title', 'album_artist',
                        'track_number', 'total_tracks', 'key', 'mode',
                        'duration_ms', 'time_signature']
    playlist_data.drop(columns = columns_to_remove, axis = 1, inplace = True)

    # Evaluate original order
    original_order_eval = evaluate_data(playlist_data_full)
    original_evals.append(original_order_eval)

    # Load and run trained model
    model = pickle.load(open('./regression/random_forest.sav', 'rb')) # change to whichever model we want to use
    playlist_data_full['order'] = model.predict(playlist_data.values)
    playlist_data_full = playlist_data_full.sort_values(by=['order'])
    playlist_data_full['order'] = np.arange(1, playlist_data_full.shape[0]+1) # convert order to integer playlist track number

    # Evaluate new order
    sorted_order_eval = evaluate_data(playlist_data_full)
    sorted_evals.append(sorted_order_eval)

    y_true = playlist_data_full['track_number']
    y_pred = playlist_data_full['order']
    mean_squared_errors.append(mean_squared_error(y_true, y_pred))

    track_eval_changes = []
    
    for metric in range(len(original_order_eval)):
        track_eval_changes.append(sorted_order_eval[metric] - original_order_eval[metric])
        
    eval_changes.append(track_eval_changes)

In [5]:
print(np.mean(mean_squared_errors))

evaluation_vector_titles = ['keyTransition', 'tempoTransition', 'trackNumTransition', 'loudenssTransition', 'timbreTransition',
                            'danceabilityTransition', 'energyTransition', 'speechinessTransition','acousticnessTransition',
                            'instrumentalnessTransition', 'livenessTransition', 'valenceTransition']
eval_changes_df = pd.DataFrame.from_records(eval_changes, columns=evaluation_vector_titles)

eval_changes_df['trackTotal'] = eval_changes_df.abs().mean(numeric_only=True, axis=1)
eval_changes_df.loc['metricTotal'] = eval_changes_df.abs().mean(numeric_only=True, axis=0)

eval_changes_df

70.00767829685994


,keyTransition,tempoTransition,trackNumTransition,loudenssTransition,timbreTransition,danceabilityTransition,energyTransition,speechinessTransition,acousticnessTransition,instrumentalnessTransition,livenessTransition,valenceTransition,trackTotal
0,0.000000,0.083333,0.000000,0.0,0.166667,-0.166667,0.250000,0.083333,0.000000,0.083333,0.166667,-0.083333,0.090278
1,0.000000,0.071429,-0.785714,0.0,0.000000,-0.214286,0.142857,0.071429,-0.071429,0.000000,0.000000,0.000000,0.113095
2,-0.043478,0.043478,-0.956522,0.0,-0.021739,0.065217,0.000000,-0.086957,0.000000,0.000000,-0.043478,-0.021739,0.106884
3,-0.173913,-0.086957,-0.869565,0.0,-0.043478,-0.043478,0.043478,0.043478,-0.086957,-0.043478,0.000000,0.043478,0.123188
4,0.000000,0.000000,-0.928571,0.0,0.000000,0.214286,-0.142857,0.142857,-0.071429,0.142857,0.214286,-0.071429,0.160714
5,-0.200000,-0.100000,-0.900000,0.0,-0.300000,0.000000,-0.100000,0.100000,-0.100000,0.000000,-0.200000,-0.300000,0.191667
6,0.000000,0.071429,-0.714286,0.0,0.000000,-0.071429,-0.214286,0.000000,0.000000,-0.071429,0.000000,0.000000,0.095238
7,-0.166667,0.083333,-0.583333,0.0,-0.083333,0.333333,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.125000
8,0.000000,-0.052632,-0.947368,0.0,0.052632,-0.157895,0.000000,-0.210526,0.000000,-0.105263,0.000000,0.052632,0.131579
9,0.041667,0.125000,-0.916667,0.0,-0.083333,-0.083333,0.083333,0.125000,0.125000,0.166667,0.000000,0.166667,0.159722


In [ ]:
# Evalute all tracks with autoencoder

encoder = tf.keras.models.load_model('./regression/encoder.h5')

x = 1
sorted_evals = []
original_evals = []
mean_squared_errors = []
eval_changes = []

for playlist_id in playlist_ids:
    
    print('Running playlist ' + playlist_id)

    # Read and set up data
    playlist_data = pd.read_csv('./data/test_set/playlist_data_' + str(x) + '.csv')
    playlist_data.drop(columns = ['Unnamed: 0'], axis = 1, inplace = True)
    playlist_data_full = playlist_data.copy(deep=True)
    x += 1

    # Remove unnecessary testing columns
    columns_to_remove = ['id', 'track_title', 'album_title', 'album_artist',
                        'track_number', 'total_tracks', 'key', 'mode',
                        'duration_ms', 'time_signature']
    playlist_data.drop(columns = columns_to_remove, axis = 1, inplace = True)

    # Evaluate original order
    original_order_eval = evaluate_data(playlist_data_full)
    original_evals.append(original_order_eval)

    # Load and run trained model
    encoded_playlist_data = encoder.predict(playlist_data)
    model = pickle.load(open('./regression/autoencoder_random_forest.sav', 'rb')) # change to whichever model we want to use
    playlist_data_full['order'] = model.predict(encoded_playlist_data)
    playlist_data_full = playlist_data_full.sort_values(by=['order'])
    playlist_data_full['order'] = np.arange(1, playlist_data_full.shape[0]+1) # convert order to integer playlist track number

    # Evaluate new order
    sorted_order_eval = evaluate_data(playlist_data_full)
    sorted_evals.append(sorted_order_eval)

    y_true = playlist_data_full['track_number']
    y_pred = playlist_data_full['order']
    mean_squared_errors.append(mean_squared_error(y_true, y_pred))

    track_eval_changes = []
    
    for metric in range(len(original_order_eval)):
        track_eval_changes.append(sorted_order_eval[metric] - original_order_eval[metric])
        
    eval_changes.append(track_eval_changes)

In [7]:
print(np.mean(mean_squared_errors))

evaluation_vector_titles = ['keyTransition', 'tempoTransition', 'trackNumTransition', 'loudenssTransition', 'timbreTransition',
                            'danceabilityTransition', 'energyTransition', 'speechinessTransition','acousticnessTransition',
                            'instrumentalnessTransition', 'livenessTransition', 'valenceTransition']
eval_changes_df = pd.DataFrame.from_records(eval_changes, columns=evaluation_vector_titles)

eval_changes_df['trackTotal'] = eval_changes_df.abs().mean(numeric_only=True, axis=1)
eval_changes_df.loc['metricTotal'] = eval_changes_df.abs().mean(numeric_only=True, axis=0)

eval_changes_df

66.19477852357146


,keyTransition,tempoTransition,trackNumTransition,loudenssTransition,timbreTransition,danceabilityTransition,energyTransition,speechinessTransition,acousticnessTransition,instrumentalnessTransition,livenessTransition,valenceTransition,trackTotal
0,-0.250000,0.000000,0.083333,0.0,0.083333,0.000000,0.000000,-0.083333,0.000000,0.000000,0.083333,-0.083333,0.055556
1,0.142857,0.142857,-0.500000,0.0,-0.071429,0.000000,-0.071429,-0.071429,-0.071429,0.000000,0.000000,0.142857,0.101190
2,-0.043478,0.000000,-0.739130,0.0,0.043478,0.173913,-0.043478,-0.108696,0.021739,0.000000,-0.086957,0.000000,0.105072
3,-0.173913,-0.086957,-0.826087,0.0,-0.130435,-0.130435,-0.043478,0.043478,-0.043478,0.000000,-0.043478,0.086957,0.134058
4,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,-0.071429,0.071429,-0.142857,0.0,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,0.000000,0.029762
7,0.083333,-0.083333,-0.166667,0.0,-0.083333,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.041667
8,0.052632,0.000000,-0.894737,0.0,0.105263,-0.052632,-0.105263,-0.157895,0.000000,-0.052632,-0.052632,0.000000,0.122807
9,-0.166667,0.000000,-0.916667,0.0,-0.208333,-0.041667,-0.041667,0.041667,0.083333,0.166667,0.000000,0.041667,0.142361


In [ ]:
# get permissions to rearrange
from spotipy.oauth2 import SpotifyOAuth
scope = "playlist-modify-public"
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope,redirect_uri='http://localhost:5678/',client_id=SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET))

# rearrange
sorted_ids = list(playlist_data_full['id'])

UPDATED_PLAYLIST = sp.playlist_replace_items(playlist_id,sorted_ids)
UPDATED_PLAYLIST